### Semana 4 - Actividad 1
### Reducción de características de un conjunto de datos para encontrar sus componentes principales y mejorar su interpretabilidad con la menor pérdida de información posible.

Lázaro Lara Martínez.
Matricula A01793198

José Mtanous Treviño.
Matricula A00169781

Ciencia y Analítica de datos.

Profesor Titular.
Jobish Vallikavungal Devassia

Profesor Tutor.
Mtro. Mario Alberto Solano Saldaña

10/Octubre/2022

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [52]:
data_url = 'https://raw.githubusercontent.com/PosgradoMNA/Actividades_Aprendizaje-/main/default%20of%20credit%20card%20clients.csv'
df = pd.read_csv(data_url)
#columns = ['amount', 'gender', 'education', 'marital_status', 'age', 'history_of_payment', '']
categorical_columns = ['X2', 'X3', 'X4', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11']


# PCA funciona bien para dimenciones numéricas, por eso vamos a quitar las dimensiones categóricas de nuesto análisis
#
df = df.drop(categorical_columns, axis=1)



In [53]:
# La columna ID la podemos remover ya que no es una dimensión
df = df.drop(['ID'],axis=1)

# Por último, removemos la etiqueta
df = df.drop(['Y'],axis=1)


# Vamos a agregarle nombres propios a las columnas se según el documento de descriptción
columnas = ['amount', 'age', 'statement1', 'statement2', 'statement3', 'statement4', 'statement5', 'statement6', 'payment1', 'payment2', 'payment3', 'payment4', 'payment5', 'payment6']

df.columns = columnas

# Revisamos los datos para ver si hay algún data faltante
df.isnull().sum()
# hay varios datos faltantes, ya que el número de renglones con datos faltantes son pocos en proporción
# los podemos eliminar de nuestro juego de datos.
x = df.dropna()

# Una vez limpio nuestro data set podemos revisar algunas de sus propiedades estadísticas
x.describe(include="all")

,amount,age,statement1,statement2,statement3,statement4,statement5,statement6,payment1,payment2,payment3,payment4,payment5,payment6
count,29971.000000,29971.000000,29971.000000,29971.000000,2.997100e+04,29971.000000,29971.000000,29971.000000,29971.000000,2.997100e+04,29971.000000,29971.000000,29971.000000,29971.000000
mean,167525.931067,35.481532,51249.461079,49203.165727,4.703742e+04,43284.028795,40332.777752,38893.714624,5663.353108,5.924604e+03,5227.418538,4828.608355,4800.375763,5219.145574
std,129732.848621,9.214731,73662.775620,71199.674796,6.937402e+04,64355.175519,60817.887131,59575.746331,16565.463372,2.305112e+04,17613.667083,15672.949501,15282.460453,17785.305817
min,10000.000000,21.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,28.000000,3564.500000,2985.500000,2.670500e+03,2331.000000,1764.500000,1256.500000,1000.000000,8.360000e+02,390.000000,297.500000,254.500000,119.500000
50%,140000.000000,34.000000,22387.000000,21207.000000,2.008900e+04,19052.000000,18117.000000,17080.000000,2100.000000,2.010000e+03,1800.000000,1500.000000,1500.000000,1500.000000
75%,240000.000000,41.000000,67220.000000,64057.000000,6.018650e+04,54572.000000,50225.000000,49238.000000,5006.500000,5.000000e+03,4509.000000,4014.000000,4037.000000,4000.000000
max,1000000.000000,79.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000


In [57]:
# Para continuear con nuestro análisis, debemos normalizar los datos 
# para esto usaremos una librería de sklearn

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
scaledData = scaler.transform(x)



In [60]:
from sklearn.decomposition import PCA
pca = PCA(n_components=14)
pca.fit(scaledData)

PCA(n_components=14)

In [61]:
x_pca = pca.transform(scaledData)


In [70]:
pcsSummary = pd.DataFrame({'Standard deviation': np.sqrt(pca.explained_variance_),
                          'Proportion of variance': pca.explained_variance_ratio_,
                          'Cumulative proportion': np.cumsum(pca.explained_variance_ratio_)#column 
                          }
                          )
pcsSummary
pcsSummary_df = pcsSummary

#pcsSummary_df['PC']= ['PC{}'.format(i) for i in range(1, len(pcsSummary_df.columns) + 1)]

row_names = np.array(['PC{}'.format(i) for i in range(1, 14 + 1)])
pcsSummary.index = row_names
pcsSummary

,Standard deviation,Proportion of variance,Cumulative proportion
PC1,2.432904,0.422773,0.422773
PC2,1.309933,0.122562,0.545335
PC3,1.022276,0.074644,0.619979
PC4,0.961860,0.066082,0.686061
PC5,0.940077,0.063123,0.749183
PC6,0.934233,0.062340,0.811523
PC7,0.882978,0.055687,0.867211
PC8,0.852435,0.051902,0.919112
PC9,0.841052,0.050525,0.969637
PC10,0.514248,0.018889,0.988526
